In [ ]:
### load data and import modules
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
import sklearn
import imblearn 


In [ ]:
### functions:
### feature importance selection 


In [ ]:
### feature_importance selection 
def best_importances(xx_train,yy_train,iterations=3,threshold=0.85):
    fo = 0
    for s in range(iterations):
        folds = StratifiedKFold(n_splits=5,shuffle=True)
        for train_x,train_y in folds.split(xx_train,yy_train):
            train_fold = xx_train.iloc[train_x].copy()
            train_fold_y = yy_train.iloc[train_x].copy()
        
            rus = RandomUnderSampler(0.5,random_state=1)
            train_fold, train_fold_y = rus.fit_resample(train_fold,train_fold_y)        
            colX = train_fold.columns
            colX=colX.drop(  ["Age","seniority_time","gross_income","first_contract_date"])
        
            od = get_dict(train_fold,train_fold_y,colX)
            xx_transform = apply_dict(train_fold,od,colX)
        
            clf = RandomForestClassifier(n_jobs=-1,n_estimators=50,class_weight="balanced")
            model_out = clf.fit(xx_transform,train_fold_y)
            fI = pd.Series(index=xx_transform.columns,data=model_out.feature_importances_)
        
            if not isinstance(fo,pd.DataFrame):
                fo = pd.DataFrame(fI)
            else:
                fo = pd.concat([fo,fI],axis=1)
    me = fo.mean(axis=1)
    me.sort_values(ascending=False,inplace=True)       
    selects = me.index[me.cumsum() < threshold]
    return(selects)


In [ ]:
### target encoder 

class Target_Encoder_I(BaseEstimator,TransformerMixin):
    """ assumes here that for estimation and prediction 
        values in X have same order; X. is changed to 
        numpy array; in case no label exists in X 
        respective na_values are being deleted"""
        
    def __init__(self,cols):
            self.cols=cols
          
            
    def fit(self,X,y):
        Xi = X.copy()
        Xi["Tar"] = y.to_numpy()
        out_dict = defaultdict(dict)
        for ii in self.cols:
            out_dict[ii] = Xi.groupby(ii)["Tar"].mean()
       
        self.out_dict = out_dict
        Xi = Xi.drop("Tar",axis=1,inplace=True)
        return self


In [ ]:
### function for custom kfolding that accounts for imbalanced datasets 
def custom_cv_3(X,y,n_splits=10,test_size=0.05,alternation1=5):
    sss = StratifiedShuffleSplit(n_splits=n_splits,random_state=110,test_size=test_size)
    for train_index, test_index in sss.split(X,y):
        mask1 = y[train_index] ==1
        targets_ix = train_index[mask1]
        remainIX = np.delete(train_index,np.where(mask1==1))

        comIX =np.random.choice(remainIX,size=int(mask1.sum()*alternation1 ),replace=False)
        new_train = np.concatenate((targets_ix,comIX))
        yield new_train, test_index

In [ ]:
### start with spltting to test/train set

##load data
df_k = pd.read_pickle("cleaned_data.pkl")
## separate targets and features
col = df_k.columns[:39].to_list() +["new_ones","leave_ones"]
X = df_k.loc[:,col].copy()
Y = df_k.loc[:,"Saving_Account_target":"Direct_Debit_target"].copy()
### split to train and validtaion set
x_train,x_validation, Y_train,Y_validation = train_test_split(X,Y,test_size=0.3,random_state=123)
### select those that have more than 300 positive cases 
boll = (Y_validation.sum(axis=0) > 300)

col_to_val = Y_validation.columns[boll]

### define output frames
output_frame = pd.DataFrame(columns={"randomforestclassifier__n_estimators","randomforestclassifier
                                     __max_depth","scores"} )
estim = list()
colnameX = list()





In [ ]:
for v in col_to_val[]:
    ### first recrods that already have the target in the basekt are removed
    to_mask = x_train[v.split("_target")[:2][0]]

    eliminate_records = x_train.index[np.where(to_mask==1)[0]]

    xx_train = x_train.copy()

    xx_train.drop(eliminate_records,inplace=True)
    
    yy_train = Y_train[v].copy()

    yy_train.drop(eliminate_records,inplace=True)

    xx_train.drop(v.split("_target")[:2][0],axis=1,inplace=True)
     
    colZ = xx_train.columns[xx_train.dtypes!="float"]  
    ### feature importance is calcualted 
    cols_to = best_importances(xx_train,yy_train,iterations=4,threshold=0.85)
    
    pickle.dump(cols_to,open(v+"Feat.pkl","wb"))
    
    colZ = xx_train[cols_to].columns[xx_train[cols_to].dtypes!="float"]
    ### pipeline is developed 
    clf = RandomForestClassifier(n_jobs=-1,class_weight="balanced")       
    
    pipeII = make_pipeline(Target_Encoder_I(cols =colZ),SimpleImputer(missing_values=np.nan,strategy="mean"),clf)
    
    parameters ={'randomforestclassifier__n_estimators':[50,100,120],'randomforestclassifier__max_depth':[5,10]}
             
    scoring2 = {"auc":make_scorer(roc_auc_score,needs_proba=True),"f":make_scorer(f1_score,average="weighted")}
    
    ks = custom_cv_3(xx_train[cols_to],yy_train,n_splits=10,test_size=0.3,alternation1=1)
    
    cc= GridSearchCV(pipeII,cv=ks,scoring=scoring2,param_grid=parameters,refit="f")
    
    cc.fit(xx_train[cols_to],yy_train)
    
    pd.DataFrame(cc.cv_results_).to_pickle("cv_res"+v+".pkl")
       
    
    ### save model
    modell = cc.best_estimator_
    pickle.dump(modell,open("best_est_"+v+".sav","wb"))
    estim.append(modell)
    pickle.dump(estim,open("estimators.sav","wb"))
    
    ## pickle.load(open(filename,"rb"))

    ok = cc.best_params_
    ok["scores"] = cc.best_score_
    colnameX.append(v)
    
    output_frame = output_frame.append(ok,ignore_index=True)
    
    output_frame.index=colnameX
    
    output_frame.to_pickle("actual_est.pkl")
    

In [ ]:
##read in estimates form CV and important features 
params = pd.read_pickle("actual_est.pkl")    

fileslist = []
for ro,dirs,files in os.walk("."):
    fileslist.append(files)
featureS = []
feat_dict = defaultdict(list)
for f in fileslist[0]:
    if "Feat" in f:
        featureS.append(f)
        #pd.read_pickle(f)
        feat_dict[f.split("Feat")[0]].extend(pd.read_pickle(f))
        

In [ ]:
### apply fit with best features and parameters to training set and test on validation set 
out_of_dict_new = defaultdict()
out_of_dict=defaultdict(list)
for vv in col_to_val:
    featS = feat_dict[vv]
    xTR = x_train[featS]
    yTR = Y_train[vv]
    
    colToV = xTR.columns[xTR.dtypes != "float"]
    #x_TR_trans_dict = get_dict(xTR,yTR,columS=colToV)
    #x_TR_trans = apply_dict(xTR,x_TR_trans_dict,colToV)    
    
    paramS = params.loc[vv]
    est = int(paramS["randomforestclassifier__n_estimators"])
    est1 = int(paramS["randomforestclassifier__max_depth"])

    clff = RandomForestClassifier(n_jobs=-1,class_weight="balanced",n_estimators=est,max_depth = est1)
    ### here pipeline differs  
    pipeII = make_pipeline(RandomUnderSampler(0.5,random_state=101),Target_Encoder_I,clff)

    model = pipeII.fit(x_TR,yTR)
    
    xVal = x_validation[featS]
    
    yVal = Y_validation[[vv]]
    
    proba_y = pipeII.predict_proba(xVal)
    
    roV = roc_auc_score(yVal,proba_y[:,1])
    out_of_dict[vv].append(roV)
    pickle.dump(out_of_dict,open("res_file.pkl","wb"))
    
    lab = pd.DataFrame(data={"True_Y":yVal,"Proba_Negative":proba_y[:,0],"Proba_Positive":proba_y[:,1]})
    lab.to_pickle(vv+".pkl")        


oo = pd.Series(out_of_dict).to_frame()
oo.columns =["Area_under_ROC"]
oo.to_csv("results.csv",sep=";",header=True)

In [ ]:
### load data files and make frame with cv results and out of bag results 

fi = list()

for m,n,p in os.walk("."):
    fi.extend(p)
odo = defaultdict()    
for h in fi:
    if "cv_res" in h:
        print(h)
        res = pd.read_pickle(h)
        odo[h] = res[["mean_test_score","std_test_score"]].mean(axis=0)
    
pd.DataFrame(odo).to_csv("means_dispersion.csv",header=True,sep=";")
cvr = pd.read_pickle("cv_resCredit_Card_target.pkl")


lims = pd.DataFrame(odo)
cl = lims.columns

newcl = [ m.split("res")[1] for m in cl.to_list()]
newcl = [m.split(".")[0] for m in newcl]
lims.columns = newcl

res = pd.read_csv("results.csv",sep=";")
type(res)#resres.s


res.iloc[:,1] =res.iloc[:,1].apply(lambda x: pd.to_numeric(x.split("[")[1].split("]")[0]))
res.columns = ["Names","AUC_out_of_bag"]
res.set_index("Names",inplace=True)
f=pd.concat([lims.T,res],axis=1)
f["upper_band"] = f["mean_test_score"] + 2*f["std_test_score"]
f["lower_band"] = f["mean_test_score"] -2 *f["std_test_score"]
fig = plt.figure(figsize=(8,8))
f[["upper_band","lower_band","AUC_out_of_bag"]].plot(figsize=(10,10),fontsize=10,rot=99,title="cv auc roc vs out of bag roc")



round(f,2).to_csv("results_2.csv",sep=";",header=True)